In [2]:
# ndim: 배열이 몇차원인지 알려줌
import numpy as np
x=np.array(1)
print(x.ndim) # 0차원

# np.array([1,2,3])은 3차원 벡터라고도 한다 (1차원이지만, 원소수에 따른 관점)

# variable class
class Variable:
    def __init__(self,data):
        self.data=data

# function class
class Function:
    def __call__(self,input): # __call__:파이썬의 특수 메서드로 f=Function()형태로 대입해두면 사용 가능
        x=input.data
        x=x**2
        return Variable(x)

x=Variable(np.array(10))
f=Function()
x=f(x)
print(x, x.data)

# function클래스는 기반클래스로 모든 함수의 공통적인 기능만 모아놓기
class Function: # Variable->Variable
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        return output
    def forward(self, x):
        raise NotImplementedError() # 구체적인것은 상속받은 클래스에서 정의

class Square(Function):
    def forward(self, x):
        return x**2

x=Variable(np.array(10))
f=Square()
y=f(x)
print(type(y),y.data)


0
<__main__.Variable object at 0x7f3aeb04e490> 100
<class '__main__.Variable'> 100


In [3]:
class Exp(Function):
  def forward(self, x):
      return np.exp(x)

# function클래스의 __call__메서드는 모두 입력, 출력이 Variable이므로 자연스럽게 이어서 쓸 수 있다
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)
print(y.data)

1.648721270700128


#수치 미분

In [5]:
# 변수, 함수 클래스 구현 이유: 자동미분 구현 위함
# 간단한 미분방법 - 수치 미분: 변화량h를 매우 작은 값으로 하드코딩(대체). 진정한 미분을 근사한다.
# 근사 오차를 줄이는 방법으로는 중앙차분이 있다.
def numerical_diff(f,x ,eps=1e-4): # 1e-4=0.0001
  x0=Variable(x.data-eps)
  x1=Variable(x.data+eps)
  y0=f(x0)
  y1=f(x1)
  return (y1.data-y0.data)/(2*eps) # 일반 데이터타입으로 리턴 Var인스턴스 아님

f=Square()
x=Variable(np.array(2.0))
dy=numerical_diff(f,x)
print(dy) # 오차값: 0.000000000004


4.000000000004


In [6]:
# 합성함수의 미분
def f(x):
  A=Square()
  B=Exp()
  C=Square()
  return C(B(A(x)))

x=Variable(np.array(0.5))
dy=numerical_diff(f,x)
print(dy) # .5에서 x를 작은 값만큼 변화시키면 y는 작은 값의 3.2배만큼 변한다.

3.2974426293330694


수치 미분은 구현하기 쉽지만
1. 오차를 포함하고
2. 계산량이 많다. (변수 각각을 미분)

# 수동 역전파


In [7]:
class Variable: # 미분값도 저장하는 Variable Type
  def __init__(self, data):
    self.data=data # np.darray
    self.grad=None # np.darray
# data, grad는 ndarray type으로 가정

class Function:
  def __call__(self, input):
    x=input.data
    y=self.forward(x)
    output=Variable(y)
    self.input=input # input을 기억
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy): # gy는 출력쪽에서 전해지는 미분값
    raise NotImplementedError()

In [8]:
class Square(Function):
  def forward(self, x):
    return x**2

  def backward(self, gy):
    x=self.input.data
    gx=2*x*gy
    return gx

class Exp(Function):
  def forward(self, x):
    return np.exp(x)

  def backward(self, gy):
    x=self.input.data
    gx=np.exp(x)*gy
    return gx

In [9]:
# 순전파
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)


In [11]:
# 역전파
y.grad=np.array(1.0)
b.grad=C.backward(y.grad)
a.grad=B.backward(b.grad)
x.grad=A.backward(a.grad)
print(x.grad)

3.297442541400256


# 자동 역전파

In [16]:
# 순전파를 해주면, 역전파까지 자동으로 이루어지는 구조
# Define-by-run: 딥러닝에서 수행하는 계산들을 계산 시점에 연결하는 방식으로 동적계산그래프 라고도 한다.
# 함수 순서를 리스트 형태로 저장
class Variable:
  def __init__(self, data):
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

class Function:
  def __call__(self, input):
    x=input.data
    y=self.forward(x)
    output=Variable(y)

    output.set_creator(self) #창조자 설정(변수에 관련 연결 기록)
    self.input=input
    self.output=output #출력도 저장
    return output
  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy): # gy는 출력쪽에서 전해지는 미분값
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    return x**2

  def backward(self, gy):
    x=self.input.data
    gx=2*x*gy
    return gx

class Exp(Function):
  def forward(self, x):
    return np.exp(x)

  def backward(self, gy):
    x=self.input.data
    gx=np.exp(x)*gy
    return gx

In [20]:
# 순전파
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

# 검증
assert y.creator==C
assert y.creator.input==b
assert y.creator.input.creator==B
assert y.creator.input.creator.input==a
assert y.creator.input.creator.input.creator==A
assert y.creator.input.creator.input.creator.input==x
# 계산 그래프는 함수와 변수 사이의 연결로 구성되고 연결은 실제 계산 수행 시점에 발생한다. Define-by-run

In [25]:
y.grad=np.array(1.0)
C=y.creator
b=C.input
b.grad=C.backward(y.grad)
print(b.grad)

2.568050833375483


In [27]:
B=b.creator
a=B.input
a.grad=B.backward(b.grad)

In [28]:
A=a.creator
x=A.input
x.grad=A.backward(a.grad)


In [30]:
print(x.grad)

3.297442541400256


In [31]:
# 역전파 코드 자동화
class Variable:
  def __init__(self, data):
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func
# 추가
  def backward(self):
    f=self.creator # 함수를 가져온다
    if f is not None: # f is None이면 역전파 종료
      x=f.input
      x.grad=f.backward(self.grad) # 함수의 backward 메서드 호출
      x.backward() # 하나 앞 변수의 backward 메서드 호출(재귀)


In [33]:
# 순전파
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

y.grad=np.array(1.0)
y.backward() # 역전파
print(x.grad)

3.297442541400256


# 재귀에서 반복문으로 (처리 효율 업)


In [40]:
# 역전파 코드 자동화
class Variable:
  def __init__(self, data):
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func
# 추가
  def backward(self):
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      x,y= f.input, f.output # 함수 입출력 가져옴
      x.grad=f.backward(y.grad) #self.grad 하면 안됨(1.0만 호출) y.grad해야 실제 f.output계속 호출

      if x.creator is not None:
        funcs.append(x.creator) # 하나 앞 변수의 함수를 리스트에 추가



In [41]:
# 순전파
A=Square()
B=Exp()
C=Square()

x=Variable(np.array(0.5))
a=A(x)
b=B(a)
y=C(b)

y.grad=np.array(1.0)
y.backward() # 역전파
print(x.grad)

3.297442541400256


# 파이썬 함수로 이용

In [42]:
# 1) 클래스 선언과 argument전달의 불편 해소 : 그냥 파이썬 함수로 만들기

def square(x):
  f=Square() # 클래스를 함수 안에서 사용
  return f(x)

def exp(x):
  f=Exp() # 클래스를 함수 안에서 사용
  return f(x)

# or..
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

In [43]:
x=Variable(np.array(0.5))
a=square(x)
b=exp(a)
y=square(b)
y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [44]:
# 개선
x=Variable(np.array(0.5))
y=square(exp(square(x)))
y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [45]:
# 2) backward 메소드 간소화 - y.grad=np.array(1.0)생략
class Variable:
  def __init__(self, data):
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None: # 추가
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      x,y= f.input, f.output # 함수 입출력 가져옴
      x.grad=f.backward(y.grad) #self.grad 하면 안됨(1.0만 호출) y.grad해야 실제 f.output계속 호출

      if x.creator is not None:
        funcs.append(x.creator) # 하나 앞 변수의 함수를 리스트에 추가




In [46]:
x=Variable(np.array(0.5))
y=square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


In [47]:
# 3) ndarray만 취급하기
class Variable:
  def __init__(self, data):
    # 추가
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError("{}은 지원 불가".format(type(data)))
    self.data=data
    self.grad=None
    self.creator=None

  def set_creator(self, func):
    self.creator=func

  def backward(self):
    if self.grad is None:
      self.grad=np.ones_like(self.data)
    funcs=[self.creator]
    while funcs:
      f=funcs.pop() # 함수 가져옴
      x,y= f.input, f.output # 함수 입출력 가져옴
      x.grad=f.backward(y.grad) #self.grad 하면 안됨(1.0만 호출) y.grad해야 실제 f.output계속 호출

      if x.creator is not None:
        funcs.append(x.creator) # 하나 앞 변수의 함수를 리스트에 추가

In [49]:
x=Variable(np.array(1.0))
x=Variable(None)

# x=Variable(1.0) TypeError: <class 'float'>은 지원 불가
